In [ ]:
import os
from safetensors.torch import load_file


dir_path = "local/qwen3-4b/Qwen/Qwen3-4B-4bit-qat"
state_dict = {}
for fname in sorted(os.listdir(dir_path)):
    if fname.endswith(".safetensors"):
        state_dict.update(load_file(os.path.join(dir_path, fname)))

In [ ]:
next(iter(state_dict.values())).dtype

In [ ]:
import torch
from paretoq_qat import get_quantized_model_from_qat_state_dict

model = get_quantized_model_from_qat_state_dict(
    state_dict, 
    "Qwen/Qwen3-4b", 
    nbits=4,
    torch_dtype=torch.float32,
    device_map="cpu",
)

In [ ]:
from transformers import AutoModelForCausalLM  # type: ignore

model1 = AutoModelForCausalLM.from_pretrained(
    "test",
    torch_dtype=torch.bfloat16,
    device_map="cpu",
)

model2 = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-0.6B",
    torch_dtype=torch.bfloat16,
    device_map="cpu",
)

replace_linear_with_quantized_linear(model2.model, w_bits=4)

input_ids = torch.arange(32)[None]

with torch.no_grad():
    out1 = model1(input_ids)
    out2 = model2(input_ids)

torch.allclose(out1.logits, out2.logits)